## Introduction

Function decorators let us “mark” functions in the source code to enhance their behavior in some way. This is powerful stuff, but mastering it requires understanding closures — which is what we get when functions capture variables defined outside of their bodies.

To understand how decorators work, we will cover:  
- How Python evaluates decorator syntax
- How Python decides whether a variable is local
- Why closures exist and how they work
- What problem is solved by nonlocal

This knowledge will allow us to:
- Implementing a well-behaved decorator
- Powerful decorators in the standard library: `@cache`, `@lru_cache`, and `@single dispatch`
- Implementing a parameterized decorator

## Decorators 101

A decorator is a callable that takes another function as an argument (the decorated function). A decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object

A decorator usually replaces a function with a different one

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()  # Invoking the decorated target actually runs inner.

running inner()


In [3]:
target  # Inspection reveals that target is a now a reference to inner.

<function __main__.deco.<locals>.inner()>

Strictly speaking, decorators are just syntactic sugar. As we just saw, you can always simply call a decorator like any regular callable, passing another function. Three essential facts make a good summary of decorators:
- A decorator is a function or another callable.
- A decorator may replace the decorated function with a different one.
- Decorators are executed immediately when a module is loaded.

## When Python Executes Decorators
**A key feature of decorators is that they run right after the decorated function is
defined. That is usually at import time (i.e., when a module is loaded by Python).**

In [5]:
registry = []
def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')

print('registry ->', registry)
f1()
f2()
f3()
main()

running register(<function f1 at 0x7ff0cf5b04a0>)
running register(<function f2 at 0x7ff0cf5b0b80>)
registry -> [<function f1 at 0x7ff0cf5b04a0>, <function f2 at 0x7ff0cf5b0b80>]
running f1()
running f2()
running f3()
running main()


The main point is to emphasize that *function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked*. This highlights the difference between what Pythonistas call import time and runtime.

The example is unusual in 2 ways:
- *The decorator function is defined in the same module as the decorated functions. **A real decorator is usually defined in one module and applied to functions in other modules**.*
- *The register decorator returns the same function passed as an argument. **In practice, most decorators define an inner function and return it to replace the decorated function.***

## Variable Scope Rules

In [6]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

When Python compiles the body of the function, it decides that `b` is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch `b` from the local scope. Later, when the call `f2(3)` is made, the body of `f2` fetches and prints the value of the local variable `a`, but when trying to fetch the value of local variable `b`, it discovers that `b` is unbound.  
This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local.

This is much better than the behavior of JavaScript, which does not require variable declarations either, but if you do forget to declare that a variable is local (with `var`), you may clobber a global variable without knowing.

If we want the interpreter to treat `b` as a global variable and still assign a new value to it within the function, we use the `global` declaration:

In [8]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)

b = 9

f3(2)

2
9


## Closures

*It does not matter whether the function is anonymous or not; what matters is that it can access nonglobal variables that are defined outside of its body*. Let's learn this through an example of a function that calculates the avg of all numbers so far:
```
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```
Where does `avg` come from, and where does it keep the history of previous values?

**Method 1: Implement with a class**

In [10]:
class Averager():
    
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
print(f"{avg(10) = }")
print(f"{avg(11) = }")
print(f"{avg(12) = }")

avg(10) = 10.0
avg(11) = 10.5
avg(12) = 11.0


The `avg` of the `Averager` class keeps the history in the `self.series` instance attribute

**Method 2: Using a higher-order function**

In [14]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

When invoked, `make_averager` returns an averager function object. Each time an averager is called, it appends the passed argument to the series, and computes the current average

In [15]:
avg = make_averager()
print(f"{avg(10) = }")
print(f"{avg(11) = }")
print(f"{avg(12) = }")

avg(10) = 10.0
avg(11) = 10.5
avg(12) = 11.0


Note that `series` is a local variable of `make_averager` because the assignment `series = []` happens in the body of that function. But when `avg(10)` is called, `make_averager` has already returned, and its local scope is long gone.

**Within `averager`, `series` is a free variable. This is a technical term meaning a variable that is not bound in the local scope**

<img src="../images/2.png" style="width: 80%;">


In [19]:
print(f"{avg.__code__.co_varnames = }")
print(f"{avg.__code__.co_freevars = }")

avg.__code__.co_varnames = ('new_value', 'total')
avg.__code__.co_freevars = ('series',)


The value for series is kept in the `__closure__` attribute of the returned function `avg`. Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_freevars`.

In [20]:
avg.__closure__

(<cell at 0x7ff0d4243700: list object at 0x7ff0ceb3d7c0>,)

In [22]:
avg.__closure__[0].cell_contents

[10, 11, 12]

**To summarize**: ***a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.***
Note that the only situation in which a function may need to deal with external variables that are nonglobal is when it is nested in another function and those variables are part of the local scope of the outer function.

## The `nonlocal` Declaration

The code below won't work since the statement `count += 1` actually means the same as `count = count + 1`

In [24]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

To work around this, the `nonlocal` keyword was introduced in Python 3. It lets you declare a variable as a free variable even when it is assigned within the function

In [25]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

### Variable Lookup Logic

When a function is defined, the Python bytecode compiler determines how to fetch a variable `x` that appears in it, based on these rules:
- If there is a global `x` declaration, `x` comes from and is assigned to the `x` global
variable module
- If there is a `nonlocal x` declaration, `x` comes from and is assigned to the `x` local variable of the nearest surrounding function where `x` is defined.
- If `x` is a parameter or is assigned a value in the function body, then `x` is the local
variable.
- If `x` is referenced but is not assigned and is not a parameter:  
    - `x` will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes).   
    - If not found in surrounding scopes, it will be read from the module global scope.  
    - If not found in the global scope, it will be read from `__builtins__.__dict__`.  

## Implementing a Simple Decorator

Below is a simple decorator to show the running time of functions

In [27]:
import time

def clock(func):
    def clocked(*args):  # Define inner function clocked to accept any number of positional arguments
        t0 = time.perf_counter()
        result = func(*args)  # This line only works because the closure for `clocked` encompasses the `func` free variable.
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked  # Return the inner function to replace the decorated function.

Let's use the decorator

In [28]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12309731s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000047s] factorial(1) -> 1
[0.00002002s] factorial(2) -> 2
[0.00002968s] factorial(3) -> 6
[0.00003862s] factorial(4) -> 24
[0.00004699s] factorial(5) -> 120
[0.00005780s] factorial(6) -> 720
6! = 720


Remember that this code:
```python
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)
```
actually does this:
```python
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial = clock(factorial)
```

So, `clock` gets the `factorial` function as its func argument. It then creates and returns the `clocked` function, which the Python interpreter assigns to `factorial`

The `clock` decorator implemented above has a few shortcomings: it does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function

In [ ]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

## Decorators in the Standard Library

Some of the most interesting decorators in the standard library are `cache`, `lru_cache`, and `singledispatch` — all from the `functools` module

### Memoization with `functools.cache`

The `functools.cache` decorator implements memoization - an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments

Below is a very costly recursive way to compute the nth number in the Finobacci series 

In [30]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000067s] fibonacci(0) -> 0
[0.00000068s] fibonacci(1) -> 1
[0.00011204s] fibonacci(2) -> 1
[0.00000035s] fibonacci(1) -> 1
[0.00000039s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00002204s] fibonacci(2) -> 1
[0.00004297s] fibonacci(3) -> 2
[0.00017880s] fibonacci(4) -> 3
[0.00000035s] fibonacci(1) -> 1
[0.00000024s] fibonacci(0) -> 0
[0.00000032s] fibonacci(1) -> 1
[0.00001994s] fibonacci(2) -> 1
[0.00003956s] fibonacci(3) -> 2
[0.00000030s] fibonacci(0) -> 0
[0.00000041s] fibonacci(1) -> 1
[0.00002016s] fibonacci(2) -> 1
[0.00000035s] fibonacci(1) -> 1
[0.00000025s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00002151s] fibonacci(2) -> 1
[0.00004157s] fibonacci(3) -> 2
[0.00008503s] fibonacci(4) -> 3
[0.00014434s] fibonacci(5) -> 5
[0.00034718s] fibonacci(6) -> 8
8


We can see that `fibonacci(1)` is called eight times, `fibonacci(2)` five times.  

Below is a faster way to implement it using caching:

In [31]:
import functools

@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000066s] fibonacci(0) -> 0
[0.00000058s] fibonacci(1) -> 1
[0.00012059s] fibonacci(2) -> 1
[0.00000130s] fibonacci(3) -> 2
[0.00014710s] fibonacci(4) -> 3
[0.00000095s] fibonacci(5) -> 5
[0.00017158s] fibonacci(6) -> 8
8


All the arguments taken by the decorated function must be hashable, because the underlying `lru_cache` uses a dict to store the results, and the keys are made from the positional and keyword arguments used in the calls.

Using `cache`, the `fibonacci` function is called only once for each value of `n`. 

Besides making silly recursive algorithms viable, `@cache` really shines in applications that need to fetch information from remote API

### Using `lru_cache`

The `functools.cache` decorator is actually a simple wrapper around the older func
`tools.lru_cache` function, which is more flexible and compatible with Python 3.8 and earlier versions.  

The main advantage of `@lru_cache` is that its memory usage is bounded by the maxsize parameter, which has a rather conservative default value of 128—which means the cache will hold at most 128 entries at any time.

```python
@lru_cache(maxsize=2**20, typed=True)
def costly_function(a, b):
    ...
```

## Single Dispatch Generic Functions

Imagine we are creating a tool to debug web applications. We want to generate HTML displays for different types of Python objects.

In [32]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [33]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [34]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [35]:
htmlize(42)

'<pre>42</pre>'

In [36]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [37]:
htmlize(True)

'<pre>True</pre>'

This works for any Python type, but now we want to extend it to generate custom displays for some types? The `functools.singledispatch` decorator allows different modules to contribute to
the overall solution, and lets you easily provide specialized functions even for types
that belong to third-party packages that you can’t edit.

In [38]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers


@singledispatch  # marks the base function that handles the object type.
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'


@htmlize.register  # Each specialized function is decorated with @«base».register
def _(text: str) -> str:  # The type of the first argument given at runtime determines when this particular function definition will be used
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'


@htmlize.register  # For each additional type to get special treatment, register a new function with a matching type hint in the first parameter
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'


@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'


@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'


@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'


@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'


In [39]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [40]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [41]:
print(htmlize([1, 2, 3]))

<ul>
<li><pre>1 (0x1)</pre></li>
<li><pre>2 (0x2)</pre></li>
<li><pre>3 (0x3)</pre></li>
</ul>


## Parameterized Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function. So how do you make a decorator accept other arguments? The answer is: **make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated**.

In [45]:
registry = set()  # registry is now a set, so adding and removing functions is faster.


def register(active=True):  # register takes an optional keyword argument.
    
    def decorate(func):
        print('running register'
            f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func  # Because decorate is a decorator, it must return a function.

    return decorate  # register is our decorator factory, so it returns decorate.


@register(active=False)
def f1():
    print('running f1()')


@register()
def f2():
    print('running f2()')


def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7ff0ceb4f420>)
running register(active=True)->decorate(<function f2 at 0x7ff0ceb4f4c0>)


Note how only the `f2` function appears in the `registry`; `f1` does not appear because `active=False` was passed to the register decorator factory, so the decorate that was applied to `f1` did not add it to the registry.

In [46]:
registry

{<function __main__.f2()>}

We can use `register` as a regular function

In [48]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7ff0ceb4e0c0>)


<function __main__.f3()>

### The Parameterized Clock Decorator

In [49]:
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)


for i in range(3):
    snooze(.123)

[0.12309443s] snooze(0.123) -> None
[0.12309452s] snooze(0.123) -> None
[0.12314299s] snooze(0.123) -> None


In [50]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12319976300932467s
snooze: 0.12310446699848399s
snooze: 0.12317454499134328s


In [51]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


### A Class-Based Clock Decorator
As a final example, let's implement a parameterized clock decorator implemented as a class with `__call__`.

In [52]:
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock_class:  # Instead of a clock outer function, the clock class is our parameterized decorator factory
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [53]:
@clock_class('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


## Chapter Summary

- We started with a simple `@register` decorator without an inner function, and finished with a parameterized `@clock()` involving two levels of nested functions.
- Understanding how decorators actually work required covering the differenc between import time and runtime, then diving into variable scoping, closures, and the new nonlocal declaration. Mastering closures and nonlocal is valuable not only to build decorators, but also to code event-oriented programs for GUIs or asynchronous I/O with callbacks, and to adopt a functional style when it makes sense.
- Parameterized decorators almost always involve at least two nested functions, maybe more if you want to use `@functools.wraps` to produce a decorator that provides better support for more advanced techniques.